In [349]:
'''
IMPORTS
'''

#Imports the pandas library to read and process data tables
import pandas as pd

#Imports numpy library to perform mathematical operations on Arrays
import numpy as np

#Imports the StandardScalar from sklearn model to normalize (scale) the features
from sklearn.preprocessing import StandardScaler

#Imports the train_test_split class from sklearn model to split the dataset
from sklearn.model_selection import train_test_split

#Imports the Decision Tree Classifier from sklearn tree
from sklearn.tree import DecisionTreeClassifier

#Imports the accuracy_score class from sklearn metrics to test accuracy between acutal and predicted data
from sklearn.metrics import accuracy_score

In [351]:
#Creates a pandas Dataframe by reading the list of songs dataset from a local CSV File
df = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding = 'latin-1')

In [353]:
df[0:5]

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,1/12/2023,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,5/31/2024,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


In [355]:
#Some columns have comma ',' so we must remove the commas before using the data for training and testing

change_col = ['Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach', 
              'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 
              'TikTok Views', 'YouTube Playlist Reach', 'AirPlay Spins', 'SiriusXM Spins',
              'Deezer Playlist Reach', 'Pandora Streams', 'Pandora Track Stations', 
              'Soundcloud Streams', 'Shazam Counts']

for remove_comma in change_col:
    df[remove_comma] = df[remove_comma].str.replace(',', '')

df.fillna(0, inplace=True)
    
df[change_col].head(5)

,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,YouTube Playlist Reach,AirPlay Spins,SiriusXM Spins,Deezer Playlist Reach,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts
0,390470936,30716,196631588,84274754,1713126,5767700,651565900,5332281936,150597040,40975,684,17598718,18004655,22931,4818457,2669262
1,323703884,28113,174597137,116347040,3486739,674700,35223547,208339025,156380351,40778,3,10422430,7780028,28444,6623075,1118279
2,601309283,54331,211607669,122599116,2228730,3025400,275154237,3369120610,373784955,74333,536,36321847,5022621,5639,7208651,5285340
3,2031280633,269802,136569078,1096100899,10629796,7189811,1078757968,14603725994,3351188582,1474799,2182,24684248,190260277,203384,0,11822942
4,107034922,7223,151469874,77373957,3670188,16400,0,0,112763851,12185,1,17660624,4493884,7006,207179,457017


In [357]:
#Let's set up a feature matrix and a label vector for the dataset

feature_columns = ['Track Score', 'Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach', 'Spotify Popularity', 'YouTube Views',	
                   'YouTube Likes',	'TikTok Posts',	'TikTok Likes',	'TikTok Views',	'YouTube Playlist Reach', 'Apple Music Playlist Count',	
                   'AirPlay Spins',	'SiriusXM Spins', 'Deezer Playlist Count', 'Deezer Playlist Reach',	'Amazon Playlist Count', 
                   'Pandora Streams', 'Pandora Track Stations',	'Soundcloud Streams', 'Shazam Counts', 'Explicit Track']

#X will be the feature matrix (dataframe) that stores all of our features' values
X = df[feature_columns]

#Y will be the label vector (dataframe) that will store all of the label values
y = df['All Time Rank']

In [335]:
#Creates an StandardScalar object to normalize (scale) the X train and test values
#scaler = StandardScaler()

#X = scaler.fit_transform(X)

In [359]:
#Splits up the dataset into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [361]:
print(X_train)

      Track Score Spotify Streams Spotify Playlist Count  \
1898         33.4         6576855                    307   
1370         40.1       264048274                  62386   
3038         25.2        32433183                   3830   
2361         29.4       231745345                  83611   
156         121.9       922010059                 107189   
...           ...             ...                    ...   
4426         19.9        27480893                   5435   
466          71.4      2226869580                 311766   
3092         24.9        76642740                   6264   
3772         22.1       153478699                  23576   
860          52.6       819779577                 161131   

     Spotify Playlist Reach  Spotify Popularity YouTube Views YouTube Likes  \
1898                  62869                 0.0      66986452        456000   
1370               29271052                65.0     170657868       1096724   
3038                 877033               

In [365]:
'''
KEYVAN'S SECTION
'''

#Instantiates Decision Tree Classifer object
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=5)

#Trains the DT object
decision_tree.fit(X_train, y_train)

#Tests DT object on testing data 
y_predict = decision_tree.predict(X_test)

#Checks the accuracy of the actual values compared to the predicted values for DT classifier
dt_accuracy = accuracy_score(y_test, y_predict)

print(dt_accuracy)

0.0010869565217391304
